In [63]:
import pandas as pd
import numpy as np
import sklearn
from sklearn import preprocessing 
import regex as re

In [64]:
#!pip install regex

In [67]:
df=pd.read_csv("../input/competitive-data-science-predict-future-sales/sales_train.csv")
df.head()
df=df.drop(columns=['date_block_num'], axis=1)
df['item_id'].fillna((df['item_id'].mean()), inplace=True)
df['item_id'].fillna((df['item_id'].mean()), inplace=True) 
df['item_price'].fillna((df['item_price'].mean()), inplace=True) 
df['item_cnt_day'].fillna((df['item_cnt_day'].mean()), inplace=True) 

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn import neighbors
from sklearn.preprocessing import OneHotEncoder
from xgboost import XGBClassifier

In [ ]:

def add_datepart(df, fldname, drop=True, time=False):
    "Helper function that adds columns relevant to a date."
    fld = df[fldname]
    fld_dtype = fld.dtype
    if isinstance(fld_dtype, pd.core.dtypes.dtypes.DatetimeTZDtype):
        fld_dtype = np.datetime64

    if not np.issubdtype(fld_dtype, np.datetime64):
        df[fldname] = fld = pd.to_datetime(fld, infer_datetime_format=True)
    targ_pre = re.sub('[Dd]ate$', '', fldname)
    attr = ['Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear',
            'Is_month_end', 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start']
    if time: attr = attr + ['Hour', 'Minute', 'Second']
    for n in attr: df[targ_pre + n] = getattr(fld.dt, n.lower())
    df[targ_pre + 'Elapsed'] = fld.astype(np.int64) // 10 ** 9
    if drop: df.drop(fldname, axis=1, inplace=True)

In [ ]:
add_datepart(df,"date")

In [ ]:
df.head()

In [ ]:
df['Is_month_end']


In [ ]:
label_encoder = preprocessing.LabelEncoder()


In [ ]:
df['Is_month_end']= label_encoder.fit_transform(df['Is_month_end']) 
df['Is_month_start']= label_encoder.fit_transform(df['Is_month_start']) 
df['Is_quarter_end']= label_encoder.fit_transform(df['Is_quarter_end']) 
df['Is_quarter_start']= label_encoder.fit_transform(df['Is_quarter_start']) 
df['Is_year_end']= label_encoder.fit_transform(df['Is_year_end']) 
df['Is_year_start']= label_encoder.fit_transform(df['Is_year_start'])

In [ ]:
item_cnt=df['item_cnt_day']
df=df.drop(labels='item_cnt_day',axis=1)

In [ ]:
df.insert(17,'item_cnt_day',item_cnt);

In [ ]:
df.head()


In [ ]:
df.iloc[0:2]


In [ ]:
(5*len(df))/100


In [ ]:
2935849-146792

In [ ]:

train_df=df[0:2789057]

In [ ]:
valid_df=df[2789057:len(df)]

In [ ]:
len(train_df),len(valid_df)

In [ ]:
y_train=train_df['item_cnt_day']
x_train=train_df.drop('item_cnt_day',axis=1)

In [ ]:

len(x_train),len(y_train)

In [ ]:
!pip install xgboost
sklearn.neighbors.KNeighborsClassifier
sklearn.linear_model.LinearRegression
sklearn.ensemble.RandomForestClassifieritem_cnt_day
sklearn.ensemble.GradientBoostingClassifier
xgb_model = xgb.XGBClassifier()

In [ ]:
model1=RandomForestClassifier()
model2=XGBClassifier()
model3=neighbors.KNeighborsClassifier()
model4=GradientBoostingClassifier()

pred1=model1.fit(x_train,y_train)
pred2=model2.fit(x_train,y_train)
pred3=model3.fit(x_train,y_train)
pred4=model4.fit(x_train,y_train)

pred1=model1.predict(valid_df)
pred2=model2.predict(valid_df)
pred3=model3.predict(valid_df)
pred4=model4.predict(valid_df)

In [ ]:
#test_pred1=model1.predict(OH_X_test)
#test_pred2=model2.predict(OH_X_test)
#test_pred3=model3.predict(OH_X_test)
#test_pred4=model4.predict(OH_X_test)

stacked=np.column_stack((pred1,pred2,pred3,pred4))
#stacked_test_predictions=np.column_stack((test_pred1,test_pred2,test_pred3,test_pred4))

meta_model=XGBClassifier()
meta_model.fit(stacked,valid_df)
